In [1]:
%load_ext autoreload
%autoreload 2

In [101]:
import nn

In [102]:
import torch as th

In [103]:
x = th.rand((5, 3))
f = nn.HyperbolicLinearLinear(3, 2)

In [104]:
f(x)

tensor([[0.0890, 0.0990],
        [0.5430, 0.5551],
        [0.5395, 0.5566],
        [0.5341, 0.5588],
        [0.3155, 0.2451]], grad_fn=<MulBackward0>)

In [6]:
from non_tensor_ops import *

In [48]:
emb_dim = 20
v1 = np_random.uniform(-0.5, 0.5, emb_dim).astype(np.float32)
v2 = np_random.uniform(-0.5, 0.5, emb_dim).astype(np.float32)
v3 = np_random.uniform(-0.5, 0.5, (10, emb_dim)).astype(np.float32)
v1 = v1 * 0.59999 / LA.norm(v1)
v2 = v2 * 0.99 / LA.norm(v2)
v3 = np.stack((v1, v2))

In [49]:
M = np.random.rand(emb_dim, 5).astype(np.float32)

In [50]:
mob_mat_mul(M, v3, 1)

Mx norm1:  1.4754727
x norm  [0.59999 0.99   ]
Norm factor:  [2.459162  1.4903765]
inner2:  [0.63435405 0.67724097]
Mx:  [[-0.00241915  0.3175991   0.22756176  0.28619313  0.29497212]
 [-0.8694726  -0.32169852 -0.83165216 -0.53890795 -0.1179212 ]]


ValueError: operands could not be broadcast together with shapes (2,) (2,5) 

In [61]:
def mob_mat_mul(M, x, c):
    Mx = x @ M
    MX_norm = LA.norm(Mx.T, axis=0)
    x_norm = LA.norm(x.T, axis=0)
    print('Mx: ', Mx)
    print('Mx norm1: ',MX_norm)
    print('x norm ', x_norm)
    sqrt_recip = 1. / np.sqrt(c)
    norm_factor = MX_norm / x_norm
    print('Norm factor: ', norm_factor)
    inner1 = np.tanh(norm_factor * np.arctanh(np.sqrt(c) * x_norm)) / MX_norm
    inner2 = sqrt_recip * inner1
    inner2 = inner2.reshape(-1, 1)
    print('inner2: ', inner2)
    print('Mx: ', Mx)
    return  inner2 * Mx

In [62]:
print('Ans1: ',mob_mat_mul(M, v1, 1))
print()
print('Ans2: ',mob_mat_mul(M, v2, 1))
print()
print('Ans3: ',mob_mat_mul(M, v3, 1))

Mx:  [-0.00241917  0.3175991   0.22756174  0.28619313  0.29497212]
Mx norm1:  0.5670754
x norm  0.59999
Norm factor:  0.9451414
inner2:  [[1.01414763]]
Mx:  [-0.00241917  0.3175991   0.22756174  0.28619313  0.29497212]
Ans1:  [[-0.00245339  0.32209236  0.2307812   0.29024209  0.29914528]]

Mx:  [-0.8694726  -0.32169852 -0.83165216 -0.538908   -0.11792122]
Mx norm1:  1.3621472
x norm  0.99
Norm factor:  1.3759062
inner2:  [[0.73312695]]
Mx:  [-0.8694726  -0.32169852 -0.83165216 -0.538908   -0.11792122]
Ans2:  [[-0.63743381 -0.23584585 -0.60970661 -0.39508798 -0.08645122]]

Mx:  [[-0.00241915  0.3175991   0.22756176  0.28619313  0.29497212]
 [-0.8694726  -0.32169852 -0.83165216 -0.53890795 -0.1179212 ]]
Mx norm1:  [0.5670754 1.3621472]
x norm  [0.59999 0.99   ]
Norm factor:  [0.9451414 1.3759062]
inner2:  [[1.0141476 ]
 [0.73312694]]
Mx:  [[-0.00241915  0.3175991   0.22756176  0.28619313  0.29497212]
 [-0.8694726  -0.32169852 -0.83165216 -0.53890795 -0.1179212 ]]
Ans3:  [[-0.00245338  0.

In [63]:
from operations import *

In [64]:
v1 = th.tensor(v1)
v2 = th.tensor(v2)
v3 = th.tensor(v3)
M = th.tensor(M)

In [99]:
def mat_mult(M, x, c=C):
    x = x + EPS
    Mx = x @ M
    MX_norm = norm(th.transpose(Mx, -1, 0), dim=0)
    x_norm = norm(th.transpose(x, -1, 0), dim=0)
    #print(Mx)
    print('Mx: ', Mx)
    print('Mx norm1: ',MX_norm)
    print('x norm ', x_norm)
    sqrt_recip = 1. / np.sqrt(c)
    norm_factor = MX_norm / x_norm
    print('Norm factor: ', norm_factor)
    inner1 = tanh(norm_factor * atanh(np.sqrt(c) * x_norm)) / MX_norm
    inner2 = sqrt_recip * inner1
    inner2 = inner2.view(-1, 1)
    print('inner2: ', inner2)
    print('Mx: ', Mx)
    result =  inner2 * Mx
    return project_hyp_vecs(result, c)


In [100]:
print('Ans1: ',mat_mult(M, v1, 1))
print()
print('Ans2: ',mat_mult(M, v2, 1))
print()
print('Ans3: ',mat_mult(M, v3, 1))

Mx:  tensor([-0.0024,  0.3176,  0.2276,  0.2862,  0.2950])
Mx norm1:  tensor([0.5671])
x norm  tensor([0.6000])
Norm factor:  tensor([0.9451])
inner2:  tensor([[1.0141]])
Mx:  tensor([-0.0024,  0.3176,  0.2276,  0.2862,  0.2950])
Ans1:  tensor([[-0.0025,  0.3221,  0.2308,  0.2902,  0.2991]])

Mx:  tensor([-0.8695, -0.3217, -0.8317, -0.5389, -0.1179])
Mx norm1:  tensor([1.3621])
x norm  tensor([0.9900])
Norm factor:  tensor([1.3759])
inner2:  tensor([[0.7331]])
Mx:  tensor([-0.8695, -0.3217, -0.8317, -0.5389, -0.1179])
Ans2:  tensor([[-0.6374, -0.2358, -0.6097, -0.3951, -0.0865]])

Mx:  tensor([[-0.0024,  0.3176,  0.2276,  0.2862,  0.2950],
        [-0.8695, -0.3217, -0.8317, -0.5389, -0.1179]])
Mx norm1:  tensor([[0.5671, 1.3621]])
x norm  tensor([[0.6000, 0.9900]])
Norm factor:  tensor([[0.9451, 1.3759]])
inner2:  tensor([[1.0141],
        [0.7331]])
Mx:  tensor([[-0.0024,  0.3176,  0.2276,  0.2862,  0.2950],
        [-0.8695, -0.3217, -0.8317, -0.5389, -0.1179]])
Ans3:  tensor([[-0.0

In [30]:
LA.norm(v3, axis=1)

array([0.59999   , 0.98999995], dtype=float32)

In [105]:
m1 = th.rand((2, 3))
m2 = th.empty(2)

In [106]:
m2

tensor([-3.7936e-38,  3.0733e-41])

In [109]:
m2 @ m1

tensor([-2.4882e-39, -3.0468e-38, -1.1694e-38])

In [110]:
np.sqrt(56000)

236.64319132398464